In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn import metrics
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import math
print(os.listdir("./"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['.ipynb_checkpoints', 'detect_peaks.py', 'evaluators.py', 'kernel1b66f5b77c.ipynb', 'load_data.py', 'logistic_regression.py', 'ml-unibuc-2019-23.zip', 'naive_bayes.py', 'perceptron.py', 'sample_submission.csv', 'signal_analysis_utils.py', 'siml', 'sk_utils.py', 'test', 'train', 'train_labels.csv', '__init__.py', '__pycache__']


In [2]:
# constants
train_root_path = './train'
train_labels_path = './train_labels.csv'
test_root_path = './test'

N = 135
f_s = 100
t_n = 1.35
T = t_n / N

train_samples = 9000
test_samples = 5000
n_features = 3
bucket_size = 135

In [3]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None)
    return np.resize(dataframe.values, (bucket_size, n_features))

# test load_file
print(load_file(os.path.join(train_root_path, '16763.csv')).shape)

# load a list of files and return as a 3d numpy array
def load_group(root_path, limit = None):
    loaded = list()
    filenames = os.listdir(root_path)
    if limit is None: 
        limit = len(filenames)
    for i, name in enumerate(filenames):
        if i >= limit: break
        data = load_file(os.path.join(root_path, name))
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = np.stack(loaded, axis=0)
    return loaded

# test load_group
print(load_group(train_root_path, limit=100).shape)

# load train labels
def load_train_labels(labels_path, train_data_root_path, limit = train_samples):
    train_labels = np.zeros(limit)
    loaded = pd.read_csv(labels_path)
    
    for i, filename in enumerate(os.listdir(train_data_root_path)):
        if i >= limit: break
        file_id = int(filename.split('.')[0], 10)
        train_labels[i] = int(loaded[loaded['id'] == file_id]['class'])
        
    return train_labels.reshape(limit, )

# test load_train_labels
print(load_train_labels(train_labels_path, train_root_path, limit = 100).shape)

# load test ids
def load_test_ids(test_root_path):
    test_ids = np.zeros(test_samples)
    
    for i, filename in enumerate(os.listdir(test_root_path)):
        file_id = int(filename.split('.')[0], 10)
        test_ids[i] = file_id
        
    return test_ids

# test load_test_ids
print(load_test_ids(test_root_path))

(135, 3)
(100, 135, 3)
(100,)
[10001. 10002. 10004. ... 23992. 23998. 24000.]


In [44]:
# load data from input
train_data = load_group(train_root_path)
train_labels = load_train_labels(train_labels_path, train_root_path) - 1
test_data = load_group(test_root_path)
test_ids = load_test_ids(test_root_path)

print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_ids.shape)

(9000, 135, 3)
(9000,)
(5000, 135, 3)
(5000,)


In [45]:
# Split train_data
# train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size=0.20)

In [37]:
from siml.sk_utils import *
from siml.signal_analysis_utils import *

# denominator = 10
# X_train, Y_train = extract_features_labels(train_data, train_labels, T, N, f_s, denominator)
# X_test, Y_test = extract_features_labels(test_data, test_labels, T, N, f_s, denominator)

In [46]:
def extract_features(data):
    feature_engin = [np.mean(data, axis=2), np.sum(data**2, axis=2)]

    new_data = []
    for feature in feature_engin:
        new_data = np.append(new_data, feature)
            
    return new_data.reshape(len(data), len(feature_engin) * bucket_size)

# train_features = extract_features(train_data)
# test_features = extract_features(test_data)
train_features = np.reshape(train_data, (train_data.shape[0], train_data.shape[1]*train_data.shape[2]))
test_features = np.reshape(test_data, (test_data.shape[0], test_data.shape[1]*test_data.shape[2]))
train_features = np.append(train_features, extract_features(train_data), axis = 1)
test_features = np.append(test_features, extract_features(test_data), axis = 1)

print(train_features.shape)
print(test_features.shape)

(9000, 675)
(5000, 675)


In [47]:
def add_gravitation_orientation():
    global train_data, test_data
    # add gravitational orientation as a feature to train_data
    train_data = np.insert(train_data, 3, 0, axis=2)
    for i in train_data:
        for j in i:
            j[3] = math.sqrt(j[0]**2 + j[1]**2 + j[2]**2)

    print(train_data.shape)

    # add gravitational orientation as a feature to test_data
    test_data = np.insert(test_data, 3, 0, axis=2)
    for i in test_data:
        for j in i:
            j[3] = math.sqrt(j[0]**2 + j[1]**2 + j[2]**2)

    print(test_data.shape)

# add_gravitation_orientation()

In [35]:
# clf = RandomForestClassifier(n_estimators=1000)
# clf.fit(X_train, Y_train)
# print("Accuracy on training set is : {}".format(clf.score(X_train, Y_train)))
# print("Accuracy on test set is : {}".format(clf.score(X_test, Y_test)))


In [38]:
dict_results = batch_classify(train_features[:1000], train_labels[:1000], test_features[:1000], test_labels[:1000])
display_dict_models(dict_results)

trained Gradient Boosting Classifier in 72.40 s
trained Random Forest in 0.16 s


C:\Users\corneliu.dumitru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\corneliu.dumitru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\corneliu.dumitru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


trained Logistic Regression in 5.63 s
trained Nearest Neighbors in 0.03 s
trained Decision Tree in 0.63 s


,classifier,train_score,test_score,train_time
0,Gradient Boosting Classifier,1.000,0.554,72.397855
1,Random Forest,1.000,0.533,0.155805
3,Nearest Neighbors,0.657,0.495,0.025310
4,Decision Tree,1.000,0.382,0.628595
2,Logistic Regression,1.000,0.325,5.631273


In [39]:
def normalize_data(train_data, test_data, type=None):
    if type == None:
        return train_data, test_data
    
    if type == 'standard':
        scaler = preprocessing.StandardScaler()
        scaler.fit(train_data)
    
    if type == 'minmax':
        scaler = preprocessing.MinMaxScaler(feature_range=(0, 1)) # (0, 1) default
        scaler.fit(train_data)
        
    if type == 'l1':
        train_data_l1 = preprocessing.normalize(train_data, norm='l1', axis=1)
        test_data_l1 = preprocessing.normalize(test_data, norm='l1', axis=1)
        
        return train_data_l1, test_data_l1
    
    if type == 'l2':
        train_data_l2 = preprocessing.normalize(train_data, norm='l2', axis=1)
        test_data_l2 = preprocessing.normalize(test_data, norm='l2', axis=1)
        
        return train_data_l2, test_data_l2
    
    train_data_scaled = scaler.transform(train_data)
    test_data_scaled = scaler.transform(test_data)
        
    return train_data_scaled, test_data_scaled

In [40]:
def svm_classifier(train_data, train_labels, test_data, C):
    modelSVM = svm.SVC(C, "linear")
    modelSVM.fit(train_data, train_labels)
    
    train_labels_predicted = modelSVM.predict(train_data)
    test_labels_predicted = modelSVM.predict(test_data)
    
    return train_labels_predicted, test_labels_predicted

def compute_accuracy(true_labels, predicted_labels):
    return (true_labels == predicted_labels).mean()

In [41]:
def evaluateSVM(train_data, train_labels, test_data, test_labels):
    C = [1e-1, 0.5, 1, 10]
    norm_types = ['standard', 'l2', 'minmax']
    accuracy = np.zeros((len(norm_types), 2, len(C)))

    for j in range(len(norm_types)):
        for i in range(len(C)):
            train_data, test_data = normalize_data(train_data, test_data, type=norm_types[j])
            train_labels_predicted, test_labels_predicted = svm_classifier(train_data, train_labels, test_data, C[i])
            
            print("Train accuracy (" + norm_types[j] + "," + str(C[i]) +"): " + str(compute_accuracy(train_labels, train_labels_predicted)))
            print("Test accuracy (" + norm_types[j] + "," + str(C[i]) +"): " + str(compute_accuracy(test_labels, test_labels_predicted)))
            
            accuracy[j, 0, i] = compute_accuracy(train_labels, train_labels_predicted)
            accuracy[j, 1, i] = compute_accuracy(test_labels, test_labels_predicted)
    
    print(accuracy)
    
evaluateSVM(train_features[:1000], train_labels[:1000], test_features[:1000], test_labels[:1000])

Train accuracy (standard,0.1): 1.0
Test accuracy (standard,0.1): 0.593
Train accuracy (standard,0.5): 1.0
Test accuracy (standard,0.5): 0.593
Train accuracy (standard,1): 1.0
Test accuracy (standard,1): 0.593
Train accuracy (standard,10): 1.0
Test accuracy (standard,10): 0.593
Train accuracy (l2,0.1): 0.371
Test accuracy (l2,0.1): 0.275
Train accuracy (l2,0.5): 0.667
Test accuracy (l2,0.5): 0.481
Train accuracy (l2,1): 0.773
Test accuracy (l2,1): 0.512
Train accuracy (l2,10): 0.988
Test accuracy (l2,10): 0.543
Train accuracy (minmax,0.1): 0.825
Test accuracy (minmax,0.1): 0.517
Train accuracy (minmax,0.5): 0.979
Test accuracy (minmax,0.5): 0.548
Train accuracy (minmax,1): 0.996
Test accuracy (minmax,1): 0.546
Train accuracy (minmax,10): 1.0
Test accuracy (minmax,10): 0.543
[[[1.    1.    1.    1.   ]
  [0.593 0.593 0.593 0.593]]

 [[0.371 0.667 0.773 0.988]
  [0.275 0.481 0.512 0.543]]

 [[0.825 0.979 0.996 1.   ]
  [0.517 0.548 0.546 0.543]]]


In [48]:
def evaluate_RandomForest(train_data, train_labels, test_data, test_labels):
    rf = RandomForestClassifier(n_estimators = 1000)
    rf.fit(train_data, train_labels)
    
    train_labels_predicted = rf.predict(train_data)
    test_labels_predicted = rf.predict(test_data)
    print("Train accuracy: ", compute_accuracy(train_labels, train_labels_predicted))
    print("Test accuracy: ", compute_accuracy(test_labels, test_labels_predicted))
    
    return test_labels_predicted

test_labels_predicted = evaluate_RandomForest(train_features, train_labels, test_features, train_labels)

Train accuracy:  1.0


C:\Users\corneliu.dumitru\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  # This is added back by InteractiveShellApp.init_path()


AttributeError: 'bool' object has no attribute 'mean'

In [ ]:
# fit a model
def evaluate_model_LSTM(trainX, trainY):
    verbose, epochs, batch_size = 0, 15, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainY.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps, n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # fit network
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
    
    # evaluate model
    _, accuracy = model.evaluate(trainX, trainY, batch_size=batch_size, verbose=0)
    print(accuracy)
    
    return model

In [ ]:
# fit and evaluate a model
def evaluate_model_ConvLSTM2D(trainX, trainy, testX, testy):
    # define model
    verbose, epochs, batch_size = 0, 150, 128
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    
    # reshape into subsequences (samples, time steps, rows, cols, channels)
    n_steps, n_length = 4, 37
    trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))

    # define model
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    print(accuracy)
    
    return model

In [ ]:
# fit and evaluate random forrest model
def evaluate_model_RandomForrest(trainX, trainY, testX, testY):
    # Create a Gaussian Classifier
    model = RandomForestClassifier(n_estimators=1000)

    # Train the model using the training sets y_pred=clf.predict(X_test)
    model.fit(trainX, trainY)
    
    # Evaluate model
    print("Accuracy on training set is : {}".format(model.score(trainX, trainY)))
    print("Accuracy on test set is : {}".format(model.score(testX, testY)))
    
    return model

In [ ]:
# evaluate a model
# model = evaluate_model_RandomForrest(X_train, Y_train.ravel(), X_test, Y_test.ravel())
# model = evaluate_model_ConvLSTM2D(train_data, train_labels, test_data, test_labels)
model = evaluate_model_ConvLSTM2D(train_data, train_labels, train_data, train_labels)

#### Test with different configs
4 features 
verbose, epochs, batch_size = 0, 25, 64      -> 0.847


3 features 
verbose, epochs, batch_size = 0, 25, 64      -> 0.8795


3 features 
verbose, epochs, batch_size = 0, 35, 128     -> 0.8845


3 features
verbose, epochs, batch_size = 1, 50, 128     -> 0.8984


3 features
verbose, epochs, batch_size = 1, 100, 128    -> 0.9115

3 features + std
verbose, epochs, batch_size = 0, 100, 128    -> 0.9044

3 features + std
verbose, epochs, batch_size = 0, 150, 128    -> 0.9194

3 features + std
bucket size = 145
verbose, epochs, batch_size = 0, 150, 128    -> 0.9439



In [ ]:
# make predictions for test_data
def predictConvLSTM(model, testX):
    # reshape into subsequences (samples, time steps, rows, cols, channels)
    n_steps, n_length, n_features = 4, 37, testX.shape[2]
    testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
    predictions = model.predict(testX)
    
    return predictions

In [ ]:
def evaluate_including_test_data():
    global model, train_data, test_data, train_labels
    
    test_labels = predictConvLSTM(model, test_data)
    print(train_data.shape)
    print(test_data.shape)
    
    print(train_labels.shape)
    print(test_labels.shape)

    train_data = np.concatenate((train_data, test_data))
    train_labels = np.concatenate((train_labels, test_labels))

    model = evaluate_model_ConvLSTM2D(train_data, train_labels, train_data, train_labels)    
    
evaluate_including_test_data()

In [ ]:
# test predict
predictions = predictConvLSTM(model, test_data)
np.argmax(predictions[0])

In [49]:
print(test_labels_predicted)

NameError: name 'test_labels_predicted' is not defined

In [ ]:
d = {'id' : [], 'class' : []}
for i in range(len(predictions)):
    d['id'].append(int(test_ids[i]))
    d['class'].append(np.argmax(predictions[i]) + 1)

print(d)

In [ ]:
# Write test_labels csv file

dataframe = pd.DataFrame(data=d)
dataframe.to_csv('test_labels.csv', index=False)